In [5]:
# 导入LangChain库中的多个模块，这些模块用于加载文档、创建向量存储、嵌入、构建提示模板、连接LLM（大型语言模型）以及管理回调  
from langchain.document_loaders import OnlinePDFLoader  # 用于从在线源加载PDF文档  
from langchain.vectorstores import Chroma  # 用于创建基于向量的文档存储  
from langchain.embeddings import GPT4AllEmbeddings  # 用于嵌入文本的模型  
from langchain import PromptTemplate  # 用于构建LLM的提示模板  
from langchain.llms import Ollama  # 特定的大型语言模型（LLM）接口  
from langchain.callbacks.manager import CallbackManager  # 用于管理回调的类  
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler  # 回调处理器，用于将输出流式传输到标准输出  
from langchain.chains import RetrievalQA  # 用于实现检索式问答的链  
  
import sys  # 用于访问与Python解释器紧密相关的变量和函数  
import os  # 提供了许多与操作系统交互的功能  

In [14]:
# 定义一个上下文管理器类，用于临时抑制标准输出和标准错误输出  
class SuppressStdout:  
    def __enter__(self):  
        # 保存原始的标准输出和标准错误输出  
        self._original_stdout = sys.stdout  
        self._original_stderr = sys.stderr  
        # 将标准输出和标准错误输出重定向到/dev/null（一个特殊的设备文件，会丢弃所有写入其中的数据）  
        sys.stdout = open(os.devnull, 'w')  
        sys.stderr = open(os.devnull, 'w')  
  
    def __exit__(self, exc_type, exc_val, exc_tb):  
        # 关闭重定向的文件对象  
        sys.stdout.close()  
        sys.stderr.close()  
        # 恢复原始的标准输出和标准错误输出  
        sys.stdout = self._original_stdout  
        sys.stderr = self._original_stderr  

In [ ]:
# load the pdf and split it into chunks
loader = OnlinePDFLoader("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001813756/975b3e9b-268e-4798-a9e4-2a9a7c92dc10.pdf")
data = loader.load()

In [ ]:
# 使用递归字符文本拆分器将文档拆分成小块  
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)  
all_splits = text_splitter.split_documents(data)  # 拆分文档  
  

In [ ]:
# 使用上下文管理器临时抑制输出，因为向量存储的创建可能会产生大量输出  
with SuppressStdout():  
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())  # 创建向量存储  
  

In [ ]:
# 无限循环，等待用户输入查询  
while True:  
    query = input("\nQuery: ")  # 获取用户输入的查询  
    if query == "exit":  # 如果用户输入"exit"，则退出循环  
        break  
    if query.strip() == "":  # 如果用户输入为空，则继续循环  
        continue  
  
    # 构建提示模板，用于指导LLM如何根据上下文和查询生成回答  
    template = """  
    Use the following pieces of context to answer the question at the end.  
    If you don't know the answer, just say that you don't know, don't try to make up an answer.  
    Use three sentences maximum and keep the answer as concise as possible.  
    {context}  
    Question: {question}  
    Helpful Answer:  
    """  
    QA_CHAIN_PROMPT = PromptTemplate(  
        input_variables=["context", "question"],  # 提示模板中的变量  
        template=template,  # 模板字符串  
    )  
  
    # 初始化Ollama模型，并设置回调管理器以将输出流式传输到标准输出  
    llm = Ollama(model="llama3.1", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))  
  
    # 使用RetrievalQA链来实现检索式问答，其中向量存储作为检索器  
    qa_chain = RetrievalQA.from_chain_type(  
        llm=llm,  # LLM实例  
        retriever=vectorstore.as_retriever(),  # 检索器  
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},  # 链类型的关键字参数，包括提示模板  
    )  
  
    # 执行检索式问答，并打印结果  
    result = qa_chain({"query": query})  # 执行查询并获取